<a href="https://colab.research.google.com/github/ilucas75/BTflow/blob/dev/wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import urllib.parse
import requests
from google.colab import files
import time
import pandas as pd

In [3]:
uploaded = files.upload()

Saving lei_cac40.csv to lei_cac40.csv


KeyError: ignored

In [0]:
df = pd.read_csv(list(uploaded.keys())[0], sep=';')

In [10]:
df

,label,country
0,BANQUE CENTRALE DE COMPENSATION,FR
1,European Central Counterparty N.V.,NL
2,MORGAN STANLEY & CO. INTERNATIONAL PLC,GB
3,GOLDMAN SACHS INTERNATIONAL,GB
4,XTX MARKETS LIMITED,GB
...,...,...
92,CAPITAL RESEARCH COMPANY,US-CA
93,LCH.CLEARNET LLC,US-DE
94,BANCA IMI S.P.A. IN FORMA ABBREVIATA ANCHE SOL...,IT
95,OLIVETREE FINANCIAL LIMITED,GB


In [28]:
r = df.iloc[3]
label = urllib.parse.quote(r['label'])
lang = r['country'].lower()
if lang == 'gb' or lang.startswith('us-'):
  lang = 'en'
print(f"https://{lang}.wikipedia.org/w/api.php?action=opensearch&search={label}")
resp = requests.get(f'https://{lang}.wikipedia.org/w/api.php?action=opensearch&search={label}')
print(resp)
print(resp.json()[1])

https://en.wikipedia.org/w/api.php?action=opensearch&search=GOLDMAN%20SACHS%20INTERNATIONAL
<Response [200]>
['Goldman Sachs International']


In [22]:
import urllib.parse
label = urllib.parse.quote('MORGAN STANLEY & CO. INTERNATIONAL PLC')
label

'MORGAN%20STANLEY%20%26%20CO.%20INTERNATIONAL%20PLC'

In [38]:
s_lang = {}
s_wiki_lang = {}
s_en = {}
s_wiki_en = {}
error = {}
for _, r in df.iterrows():
  try:
    label = urllib.parse.quote(r['label'])
    lang = r['country'].lower()
    if lang is in ['gb', 'vg'] or lang.startswith('us-'):
      lang = 'en'
    # search en
    if lang != 'en':
      print(f"https://en.wikipedia.org/w/api.php?action=opensearch&search={label}")
      resp = requests.get(f'https://en.wikipedia.org/w/api.php?action=opensearch&search={label}')
      print(resp)
      resp_json = resp.json()
      if len(resp_json[1]) != 0:
        s_en[r['label']] = resp_json[1][0]
        s_wiki_en[r['label']] = resp_json[3][0]
    # search lang
    print(f"https://{lang}.wikipedia.org/w/api.php?action=opensearch&search={label}")
    resp = requests.get(f'https://{lang}.wikipedia.org/w/api.php?action=opensearch&search={label}')
    print(resp)
    resp_json = resp.json()
    if len(resp_json[1]) != 0:
      s_lang[r['label']] = resp_json[1][0]
      s_wiki_lang[r['label']] = resp_json[3][0]
  except Exception as e:
    error[r['label']] = e
  time.sleep(0.5)

https://en.wikipedia.org/w/api.php?action=opensearch&search=BANQUE%20CENTRALE%20DE%20COMPENSATION
<Response [200]>
https://fr.wikipedia.org/w/api.php?action=opensearch&search=BANQUE%20CENTRALE%20DE%20COMPENSATION
<Response [200]>
https://en.wikipedia.org/w/api.php?action=opensearch&search=European%20Central%20Counterparty%20N.V.
<Response [200]>
https://nl.wikipedia.org/w/api.php?action=opensearch&search=European%20Central%20Counterparty%20N.V.
<Response [200]>
https://en.wikipedia.org/w/api.php?action=opensearch&search=MORGAN%20STANLEY%20%26%20CO.%20INTERNATIONAL%20PLC
<Response [200]>
https://en.wikipedia.org/w/api.php?action=opensearch&search=GOLDMAN%20SACHS%20INTERNATIONAL
<Response [200]>
https://en.wikipedia.org/w/api.php?action=opensearch&search=XTX%20MARKETS%20LIMITED
<Response [200]>
https://en.wikipedia.org/w/api.php?action=opensearch&search=LCH%20LIMITED
<Response [200]>
https://en.wikipedia.org/w/api.php?action=opensearch&search=HRTEU%20LIMITED
<Response [200]>
https://ie.w

In [39]:
error

{'Issar Limited': requests.exceptions.ConnectionError(urllib3.exceptions.MaxRetryError("HTTPSConnectionPool(host='vg.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?action=opensearch&search=Issar%20Limited (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7ff44b14c2b0>: Failed to establish a new connection: [Errno -2] Name or service not known',))"))}

In [43]:
s_wiki_lang

{'AMUNDI ASSET MANAGEMENT': '',
 'AMUNDI IRELAND LIMITED': None,
 'ARKEA DIRECT BANK': '',
 'AVIVA INVESTORS FRANCE': None,
 "BANCA D'ITALIA": '',
 'BANCA IMI S.P.A. IN FORMA ABBREVIATA ANCHE SOLO IMI S.P.A.': None,
 'BANQUE CENTRALE DE COMPENSATION': None,
 'BANQUE FEDERATIVE DU CREDIT MUTUEL': None,
 'BARCLAYS BANK IRELAND PUBLIC LIMITED COMPANY': None,
 'BARCLAYS BANK PLC': '',
 'BNP PARIBAS': '',
 'BNP PARIBAS ARBITRAGE': '',
 'BOFA SECURITIES EUROPE SA': None,
 'BOURSORAMA': '',
 'Baader Bank Aktiengesellschaft': None,
 'Bank Julius Bär & Co. AG': None,
 'Banque Pictet & Cie S.A.': None,
 'CITADEL SECURITIES GCS (IRELAND) LIMITED': None,
 'COMMERZBANK Aktiengesellschaft': '',
 'CREDIT AGRICOLE TITRES': None,
 'CREDIT INDUSTRIEL ET COMMERCIAL': '',
 'CREDIT SUISSE INTERNATIONAL': '',
 'CREDIT SUISSE SECURITIES SOCIEDAD DE VALORES SOCIEDAD ANONIMA': None,
 'Citigroup Global Markets Europe AG': None,
 'Credit Suisse AG': None,
 'DEUTSCHE BANK AKTIENGESELLSCHAFT': '',
 'ETORO (EUROPE)

In [44]:
l_en = []
l_lang = []

for _, r in df.iterrows():
  l_en.append(s_en.get(r['label'], None))
  l_lang.append(s_lang.get(r['label'], None))
df['search_lang'] = l_lang
df['search_en'] = l_en
df

[None, None, None, None, None, None, None, None, None, 'Societe Generale Ghana', None, None, None, None, None, None, None, None, 'J.P. Morgan Asset Management', None, 'UBS AG', None, None, None, None, None, None, None, None, None, None, 'Exanet', None, None, 'Kepler Cheuvreux', None, 'BNP Paribas', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'Natixis', None, None, None, None, 'Procapitalist', 'Credit Industriel et Commercial', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'Commerzbank Aktiengesellschaft', None, None, None, None, None, None, None, 'Banca d’Italia', 'Credit Suisse AG', None, None, None, None, None, None, None, None, None, None, None]


In [0]:
df['search'] = df.search_lang.combine_first(df.search_en)

In [49]:
df.count()

label          97
country        97
search_lang    22
search_en      12
search         25
dtype: int64

In [0]:
df.to_csv('lei_search.csv', sep=';')

In [0]:
for _, r in df.iterrows():
  try:
    label = urllib.parse.quote(r['label'])
    lang = r['country'].lower()
    if lang == 'gb' or lang.startswith('us-'):
      lang = 'en'

In [53]:
data_name = []
data_id = []
for _, r in df.iterrows():
  label = urllib.parse.quote(r['label'])
  lang = r['country'].lower()
  if lang in ['gb', 'vg'] or lang.startswith('us-'):
    lang = 'en'
  print(r['label'])
  resp = requests.get(f'https://www.wikidata.org/w/api.php?action=wbsearchentities&search={label}&language=en&format=json')
  print(resp)
  resp_json = resp.json()
  if len(resp_json['search']) != 0:
    data_name.append(resp_json['search'][0]['label'])
    data_id.append(resp_json['search'][0]['id'])
  else:
    data_name.append(None)
    data_id.append(None)
  time.sleep(0.5)

BANQUE CENTRALE DE COMPENSATION
<Response [200]>
European Central Counterparty N.V.
<Response [200]>
MORGAN STANLEY & CO. INTERNATIONAL PLC
<Response [200]>
GOLDMAN SACHS INTERNATIONAL
<Response [200]>
XTX MARKETS LIMITED
<Response [200]>
LCH LIMITED
<Response [200]>
HRTEU LIMITED
<Response [200]>
CREDIT SUISSE SECURITIES (EUROPE) LIMITED
<Response [200]>
J.P. MORGAN SECURITIES PLC
<Response [200]>
SOCIETE GENERALE
<Response [200]>
TOWER RESEARCH CAPITAL EUROPE B.V.
<Response [200]>
Jump Trading Europe B.V.
<Response [200]>
MORGAN STANLEY FRANCE
<Response [200]>
CREDIT SUISSE SECURITIES SOCIEDAD DE VALORES SOCIEDAD ANONIMA
<Response [200]>
CITADEL SECURITIES GCS (IRELAND) LIMITED
<Response [200]>
MERRILL LYNCH INTERNATIONAL
<Response [200]>
Morgan Stanley Europe SE
<Response [200]>
CITIGROUP GLOBAL MARKETS LIMITED
<Response [200]>
J.P. Morgan AG
<Response [200]>
SIX x-clear AG
<Response [200]>
UBS AG
<Response [200]>
UBS Europe SE
<Response [200]>
AVIVA INVESTORS GLOBAL SERVICES LIMITE

In [54]:
data_name

[None,
 None,
 None,
 'GOLDMAN SACHS INTERNATIONAL',
 None,
 None,
 'HRTEU LIMITED - SYSTEMATIC INTERNALISER',
 None,
 'J.P. MORGAN SECURITIES PLC',
 'Society General',
 None,
 None,
 None,
 None,
 None,
 'MERRILL LYNCH INTERNATIONAL',
 None,
 'CITIGROUP GLOBAL MARKETS LIMITED',
 'J.P. MORGAN AG - SYSTEMATIC INTERNALISER',
 'SIX X-CLEAR AG',
 'UBS',
 'UBS EUROPE SE - TRADING',
 None,
 None,
 'BARCLAYS CAPITAL SECURITIES LIMITED',
 'CITIGROUP GLOBAL MARKETS EUROPE AG - SYSTEMATIC INTERNALISER',
 None,
 'INSTINET EUROPE LIMITED',
 None,
 'BNP PARIBAS ARBITRAGE SNC - SYSTEMATIC INTERNALISER',
 None,
 'Exanet',
 None,
 'Deutsche Bank Aktiengesellschaft Filiale Prag, organizační složka',
 'Kepler Cheuvreux',
 'HSBC Bank',
 'BNP Paribas',
 None,
 None,
 'VIRTU FINANCIAL IRELAND LIMITED',
 None,
 'RBC EUROPE LIMITED',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'SUSQUEHANNA INTERNATIONAL SECURITIES LIMITED - SYSTEMATIC INTERNALISER',
 None,
 'Amundi',
 'Berenberg Bank',
 'Natixis

In [0]:
df['data_name'] = data_name
df['data_id'] = data_id

In [56]:
df.count()

label          97
country        97
search_lang    22
search_en      12
search         25
data_name      39
data_id        39
dtype: int64

In [0]:
df.to_csv('lei_search.csv', sep=';')